In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-29"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
21174,2024-12-29,Brasil Nbb,11:00,Brasília,Bauru,1.66,1.93,168.5,1.83,1.83,-2.5,1.97,1.71,M3xDTfwf,0.602410,0.518135,0.546448,0.546448,0.120544,0.6,0.6,NaN,NaN,271.596,205.300838,0.755905,228.026,134.994524,0.592014,154.062,230.548,245.70,367.92,0.0,0.0,0.0,0.0,0.106361,0.000000,0.099917,2.35,0.470,1.404255,0.508531,0.6,0.091469,-5.00,-1.000,-0.930000,0.460216,0.2,-0.260216
21175,2024-12-29,Brasil Nbb,11:00,Caxias do Sul,Unifacisa,2.58,1.40,153.5,1.85,1.81,4.5,2.03,1.67,rJQLRY76,0.387597,0.714286,0.540541,0.552486,0.101883,0.6,0.2,NaN,NaN,277.798,117.751400,0.423874,135.526,47.947843,0.353791,185.720,181.392,119.72,156.56,0.0,0.0,0.0,0.0,0.419289,0.015456,0.137599,-0.95,-0.190,-8.315789,0.472500,0.4,-0.072500,5.16,1.032,0.387597,0.479829,0.5,0.020171
21176,2024-12-29,Brasil Nbb,11:00,Flamengo,Mogi,1.02,10.50,159.5,1.82,1.76,-20.5,1.98,1.70,2Ncnrfhm,0.980392,0.095238,0.549451,0.568182,0.075630,0.8,0.2,NaN,NaN,110.350,19.655628,0.178121,204.130,64.353101,0.315255,96.308,292.774,90.20,287.00,0.0,0.0,0.0,0.0,1.163780,0.023702,0.107603,-1.81,-0.362,-0.055249,0.919050,0.7,-0.219050,-0.90,-0.180,-52.777778,0.239129,0.2,-0.039129
21177,2024-12-29,Brasil Nbb,11:00,Minas,Franca,1.43,2.48,160.5,1.91,1.75,-6.5,2.05,1.65,G6Z4VG8s,0.699301,0.403226,0.523560,0.571429,0.102527,0.4,0.8,NaN,NaN,93.834,8.072062,0.086025,102.960,17.790200,0.172787,95.998,107.024,92.96,118.68,0.0,0.0,0.0,0.0,0.379776,0.061824,0.152888,0.72,0.144,2.986111,0.861940,0.9,0.038060,-1.39,-0.278,-5.323741,0.721861,0.6,-0.121861
21178,2024-12-29,Brasil Nbb,11:00,Pato,Corinthians Paulista,1.70,1.89,168.5,1.91,1.75,-2.5,2.00,1.69,MPDmtYMa,0.588235,0.529101,0.523560,0.571429,0.117336,1.0,0.4,NaN,NaN,263.792,153.141219,0.580538,124.506,23.917361,0.192098,190.662,211.902,159.30,116.13,0.0,0.0,0.0,0.0,0.074847,0.061824,0.118809,-0.24,-0.048,-14.583333,0.567321,0.5,-0.067321,4.80,0.960,0.927083,0.491648,0.7,0.208352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21358,2024-12-29,Europa Wbbl Feminina,10:00,Kibirkstis Vilnius F,Siauliai F,1.01,11.20,143.5,1.85,1.85,-19.5,1.95,1.76,G67LIk2t,0.990099,0.089286,0.540541,0.540541,0.079385,0.6,0.6,NaN,NaN,114.484,62.259363,0.543826,248.628,137.416155,0.552698,92.682,433.562,85.85,219.60,0.0,0.0,0.0,0.0,1.180249,0.000000,0.072426,-0.89,-0.178,-0.056180,0.000000,0.0,0.000000,-2.89,-0.578,-17.647059,0.000000,0.0,0.000000
21359,2024-12-29,Itália Coppa Italia,14:00,San Giovanni F,Faenza F,1.04,7.85,144.5,1.88,1.71,-18.5,2.02,1.60,WCpFwZmC,0.961538,0.127389,0.531915,0.584795,0.088927,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,87.48,448.40,0.0,0.0,0.0,0.0,1.083329,0.066968,0.164080,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
21360,2024-12-29,Portugal Lpb,12:00,Benfica,Sporting CP,1.22,3.98,169.5,1.86,1.84,-10.5,1.94,1.75,U978gCWs,0.819672,0.251256,0.537634,0.543478,0.070928,0.8,0.6,NaN,NaN,99.708,13.043440,0.130816,95.582,12.728599,0.133169,107.680,96.412,91.91,88.40,0.0,0.0,0.0,0.0,0.750621,0.007644,0.072819,1.04,0.208,1.057692,0.000000,0.0,0.000000,-1.55,-0.310,-9.612903,0.000000,0.0,0.000000
21361,2024-12-29,Portugal Lpb,13:00,Oliveirense,CD Póvoa,1.13,5.15,156.5,1.86,1.84,-12.5,1.95,1.74,APQyaY8K,0.884956,0.194175,0.537634,0.543478,0.079131,0.4,0.6,NaN,NaN,179.282,99.

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,11:00,Brasil Nbb,Minas,Franca,160.5,1.91,0.5077,Over
1,11:00,Brasil Nbb,Pato,Corinthians Paulista,168.5,1.91,0.5607,Over
2,11:00,Brasil Nbb,União Corinthians,Fortaleza B.C.,154.5,1.92,0.5091,Over
3,00:00,Eua Nba,Portland Trail Blazers,Dallas Mavericks,221.5,1.84,0.8826,Over
4,00:30,Eua Nba,Los Angeles Lakers,Sacramento Kings,222.5,1.84,0.5952,Over
5,13:00,Alemanha Pro A,Trier,G. Dusseldorf,168.5,1.91,0.6117,Over
6,08:30,China Wcba Feminina,Xiamen F,Tianjin F,143.5,1.88,0.7967,Over
7,10:30,Dinamarca Liga De Basquete,Svendborg,Bears Academy,164.5,1.92,0.5702,Over
8,12:00,Europa Liga Letônia,BK Ogre,Keila Coolbet,160.5,1.92,0.5332,Over
9,11:45,Itália Série A2,Pesaro,Fortitudo Bologna,152.5,1.88,0.5697,Over
